In [19]:
from manim import *
from classes.electrodynamics_classes import *
import jupyter_capture_output
from numpy import linalg as npl

video_scene = " -v WARNING --disable_caching dipole_Scene"
image_scene = f" -v WARNING --disable_caching -r {2*427},{2*240}  -s dipole_Scene"

In [18]:
# physical natural constants (fuck this shit)
c = 1
epsilon_0 = 1

In [95]:
class Dipole(Mobject):
    def __init__(self, center = np.array([0, 0, 0]), omega = 1, amplitude = 1, **kwargs):
        super().__init__(**kwargs)
        self.center = center
        self.omega = omega
        self.amplitude = amplitude


    def get_oscillation(self, t):
        return self.center + self.amplitude * np.cos(self.omega*t) * UP


    def get_charges(self, t):
        pos_plus = self.get_oscillation(t)
        pos_minus = -pos_plus 
        plus_circle = Circle(radius = 0.25, stroke_opacity = 0, fill_color = RED, fill_opacity = 0.5).move_to(pos_plus)
        plus_text = Text(r"+", font_size = 32, weight = BOLD, color = main_color).move_to(pos_plus)
        minus_circle = Circle(radius = 0.25, stroke_opacity = 0, fill_color = BLUE, fill_opacity = 0.5).move_to(pos_minus)
        minus_text = Text(r"-", font_size = 32, weight = BOLD, color = main_color).move_to(pos_minus)
        return VGroup(plus_circle, plus_text, minus_circle, minus_text)
    

    def get_dipole_field_function(self, t):
        def dipole_field(pos):
            r = np.sqrt((pos[0]-self.center[0])**2 + (pos[1]-self.center[1])**2)              # distance to center
            rho = self.omega * r / c + 0.0000001                    # some (non-zero) constant
            
            pos_plus = self.get_oscillation(t)                      # position of the positive charge
            pos_minus = -pos_plus                                   # position of the negative charge
            
            p_vec = pos_plus - pos_minus                            # dipole moment
            n_vec = pos - self.center                               # direction vector
            # print(n_vec)

            term_1 = self.omega**3/(4*PI*epsilon_0*c**3)
            term_2 = np.cross(np.cross(n_vec, p_vec), n_vec) / rho
            term_3 = (3*n_vec*np.dot(n_vec, p_vec) - p_vec)

            term_real = term_1 * np.cos(rho-self.omega*t) * (term_3/rho**3 + term_2)
            term_imag = term_1 * np.sin(rho-self.omega*t) * (term_3/rho**2 + term_2)
            return (term_real + term_imag)[0] * RIGHT + (term_real + term_imag)[1] * UP
        return dipole_field
    

    def get_efield(self, t):
        field_func = self.get_dipole_field_function(t)
        vf = ArrowVectorField(field_func)
        return vf


In [102]:
%%capture_video --path "animations/dipole_radiation/dipole_radiation.mp4"
%%manim -qm --fps 60 $video_scene


class dipole_Scene(ThreeDScene):
    def construct(self):
        self.camera.background_color = BLACK

        CVC = Text('CVC', font_size = 12, weight = BOLD, color = WHITE, font = 'Latin Modern Sans').align_on_border(RIGHT + DOWN, buff = 0.2)
        self.add(CVC)

        dipole = Dipole(omega = 1, amplitude = 1)
        charges = dipole.get_charges(0)
        efield = dipole.get_efield(0)
        self.add(charges, efield)

        def efield_updater(field):
            t = t_tracker.get_value()
            field.become(dipole.get_efield(t))

        def charges_updater(charges):
            t = t_tracker.get_value()
            charges.become(dipole.get_charges(t))


        t_tracker = ValueTracker(0)
        charges.add_updater(charges_updater)
        efield.add_updater(efield_updater)
        self.play(t_tracker.animate.set_value(2*PI), rate_func = linear, run_time = 4)  

Output saved by overwring previous file at animations/dipole_radiation/dipole_radiation.mp4.
